In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_excel('Google Dataset.xlsx')

# Data preprocessing
data['Date'] = pd.to_datetime(data['Month Starting'], errors='coerce').dt.date
#Replacing the missing values
data['Date'][31] = pd.to_datetime('2020-05-01')
data['Date'][43] = pd.to_datetime('2019-05-01')
data['Date'][55] = pd.to_datetime('2018-05-01')

data.set_index('Date', inplace=True)
data.sort_index(ascending=True, inplace=True)
data.dropna(inplace=True)

# Scaling the 'Close' column to range (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Defining the number of time steps to look back for creating input sequences
time_steps = 30

# Creating sequences and labels for LSTM
def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(data['Close'], time_steps)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1)

# Evaluating the model
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)

print(f'Training Loss: {train_loss:.4f}')
print(f'Test Loss: {test_loss:.4f}')

y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)

y_test = y_test.reshape(-1, 1)
y_test_orig = scaler.inverse_transform(y_test)

# Calculating the accuracy metric (you can replace this with your own evaluation metric)
accuracy = 1 - np.mean(np.abs((y_test_orig - y_pred) / y_test_orig))

print(f'Accuracy: {accuracy:.2%}')